In [5]:
## testing multidark
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
import h5py
#import setup
import astropy.io.fits as fits
import csv
import pandas as pd
import h5py
#import h5py_aux
import tables
import pickle
import os
from astropy.table import Table
from astropy.coordinates import SkyCoord
import tqdm
from astropy.io import ascii

In [6]:
import time

## Running a speed test for I/O and parallelization

### Consider storing the files in different formats. 
* H5PY
* .list
* .csv
* FITS
* pickled dataframe

In [7]:
##Path to Scratch space
datadir = '/global/cscratch1/sd/zzhang13/MultiDark/z0p00/'
filename = 'mdpl2_rockstar_z0p00_25.list'
outputdir = datadir+ 'z0p00/'
print(outputdir)
print(datadir+filename)

/global/cscratch1/sd/zzhang13/MultiDark/z0p00/z0p00/
/global/cscratch1/sd/zzhang13/MultiDark/z0p00/mdpl2_rockstar_z0p00_25.list


In [8]:
## Testing the speed of importing different files. 
# Make the file a dataframe. A pickle then see how fast it reads. 

In [9]:
col_dict = {0:'scale', 1:'id', 3:'desc_id', 4:'num_prog', 5:'pid', 6:'upid', 10:'mvir', 11:'rvir', 17:'x', 18:'y', 19:'z', \
            27:'Breadth_first_ID', 28:'Depth_first_ID', 39:'M200b', 40:'M200c', 41:'M500c'}

In [10]:
start = time.time()
halo_df = pd.read_table(datadir+filename, comment='#', sep='\s+', encoding='utf-8', names=list(col_dict.values()), usecols=list(col_dict.keys()))

#Masking
mask = ~(halo_df[['mvir','M200b', 'M200c', 'M500c']] == 0).any(axis=1) ##Mask non-zero
mask &= (halo_df.pid == -1)

halo_df = halo_df[mask]

end = time.time()
print(end - start)

16.834872722625732


In [19]:
## Storing the halo file as a pickle
import _pickle as cPickle

with open('test_file.pickle', 'wb') as handle:
    pickle.dump(halo_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
## Storing the halo file as a JSON
import json
# Convert DataFrame to JSON
data = halo_df.to_json('test_file.json', orient='index')


None


In [15]:
## Read from files

In [24]:
start = start = time.time()

with open('test_file.pickle', 'rb') as handle:
    a = pickle.load(handle)
    
end = time.time()
print(end- start)

0.3406493663787842


In [22]:
a

,scale,id,desc_id,num_prog,pid,upid,mvir,rvir,x,y,z,Breadth_first_ID,Depth_first_ID,M200b,M200c,M500c
0,1.0,12696469457,-1,1,-1,-1,4.516200e+10,72.460999,927.11414,886.37781,69.22025,8257592485,8257592485,51184000000,37635000000,15054000000
1,1.0,12696469637,-1,1,-1,-1,4.516200e+10,72.460999,927.22327,875.99756,95.23396,8257592881,8257592881,57206000000,42151000000,28603000000
2,1.0,12696469691,-1,1,-1,-1,4.516200e+10,72.460999,930.71973,893.91699,69.11779,8257593128,8257593128,52689000000,40646000000,37635000000
3,1.0,12696469698,-1,1,-1,-1,4.516200e+10,72.460999,916.72882,889.15527,86.82028,8257593176,8257593176,46668000000,22581000000,9032500000
4,1.0,12696470176,-1,1,-1,-1,4.516200e+10,72.460999,929.67413,872.45435,62.90503,8257594197,8257594197,46668000000,37635000000,28603000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2388197,1.0,12706613698,-1,1,-1,-1,3.010800e+09,29.382000,946.57672,961.71594,976.55042,10134789166,10134789166,3010800000,3010800000,1505400000
2388198,1.0,12706613779,-1,1,-1,-1,3.010800e+09,29.382000,996.28320,986.23865,990.82837,10134789171,10134789171,21076000000,3010800000,1505400000
2388199,1.0,12706613796,-1,1,-1,-1,3.010800e+09,29.382000,989.24725,970.77338,989.12335,10134789185,10134789185,16560000000,3010800000,3010800000
2388214,1.0,12706617625,-1,1,-1,-1,3.010800e+09,29.382000,947.82330,959.62164,973.51508,10134789395,10134789395,10538000000,3010800000,3010800000


In [25]:
start = time.time()
b = pd.read_json('test_file.json')
end = time.time()
print(end-start)

1101.9641182422638


In [ ]:
b